# Fit Functions

[Fit functions]: ../../api_static/plasmapy.analysis.fit_functions.rst
[linregress()]: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html
[curve_fit()]: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html
[fsolve()]: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fsolve.html

[Fit functions] are a set of callable classes designed to aid in fitting analytical functions to data.  A fit function class combines the following functionality:

1. An analytical function that is callable with given parameters or fitted parameters.
1. Curve fitting functionality (usually SciPy's [curve_fit()] or [linregress()]), which stores the fit statistics and parameters into the class.  This makes the function easily callable with the fitted parameters.
1. Error propagation calculations.
1. A root solver that returns either the known analytical solutions or uses SciPy's [fsolve()] to calculate the roots.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

from pathlib import Path

from plasmapy.analysis import fit_functions as ffuncs

plt.rcParams["figure.figsize"] = [10.5, 0.56 * 10.5]

## Contents:

1. [Fit function basics](#Fit-function-basics)
1. [Fitting to data](#Fitting-to-data)
    1. [Getting fit results](#Getting-fit-results)
    1. [Fit function is callable](#Fit-function-is-callable)
    1. [Plotting results](#Plotting-results)
    1. [Root solving](#Root-solving)

## Fit function basics

[fit functions]: ../../api_static/plasmapy.analysis.fit_functions.rst
[ExponentialPlusLinear]: ../../api/plasmapy.analysis.fit_functions.ExponentialPlusLinear.rst

There is an ever expanding collection of [fit functions], but this notebook will use [ExponentialPlusLinear] as an example.

A fit function class has no required arguments at time of instantiation.

In [ ]:
# basic instantiation
explin = ffuncs.ExponentialPlusLinear()

# fit parameters are not set yet
(explin.params, explin.param_errors)

Each fit parameter is given a name.

In [ ]:
explin.param_names

These names are used throughout the [fit function's documentation](../../api/plasmapy.analysis.fit_functions.ExponentialPlusLinear.rst), as well as in its `__repr__`, `__str__`, and `latex_str` methods.

In [ ]:
(explin, explin.__str__(), explin.latex_str)

## Fitting to data

[curve_fit()]: ../../api/plasmapy.analysis.fit_functions.ExponentialPlusLinear.rst#plasmapy.analysis.fit_functions.ExponentialPlusLinear.curve_fit
[linregress()]: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html
[Linear]: ../../api/plasmapy.analysis.fit_functions.Linear.rst#plasmapy.analysis.fit_functions.Linear.curve_fit

Fit functions provide the [curve_fit()] method to fit the analytical function to a set of $(x, y)$ data.  This is typically done with SciPy's [curve_fit()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html) function, but fitting is done with SciPy's [linregress()] for the [Linear] fit function.

Let's generate some noisy data to fit to...

In [ ]:
params = (5.0, 0.1, -0.5, -8.0)  # (a, alpha, m, b)
xdata = np.linspace(-20, 15, num=100)
ydata = explin.func(xdata, *params) + np.random.normal(0.0, 0.6, xdata.size)

plt.plot(xdata, ydata)
plt.xlabel("X", fontsize=14)
plt.ylabel("Y", fontsize=14)

[curve_fit()]: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html

The fit function `curve_fit()` shares the same signature as SciPy's [curve_fit()], so any `**kwargs` will be passed on.  By default, only the $(x, y)$ values are needed.

In [ ]:
explin.curve_fit(xdata, ydata)

### Getting fit results

After fitting, the fitted parameters, uncertainties, and [coefficient of determination](https://en.wikipedia.org/wiki/Coefficient_of_determination), or $r^2$, values can be retrieved through their respective properties, `params`, `parame_errors`, and `rsq`.

In [ ]:
(explin.params, explin.params.a, explin.params.alpha)

In [ ]:
(explin.param_errors, explin.param_errors.a, explin.param_errors.alpha)

In [ ]:
explin.rsq

### Fit function is callable

Now that parameters are set, the fit function is callable.

In [ ]:
explin(0)

Associated errors can also be generated.

In [ ]:
y, y_err = explin(np.linspace(-1, 1, num=10), reterr=True)
(y, y_err)

Known uncertainties in $x$ can be specified too.

In [ ]:
y, y_err = explin(np.linspace(-1, 1, num=10), reterr=True, x_err=0.1)
(y, y_err)

### Plotting results

In [ ]:
# plot original data
plt.plot(xdata, ydata, marker="o", linestyle=" ", label="Data")
ax = plt.gca()
ax.set_xlabel("X", fontsize=14)
ax.set_ylabel("Y", fontsize=14)

ax.axhline(0.0, color="r", linestyle="--")

# plot fitted curve + error
yfit, yfit_err = explin(xdata, reterr=True)
ax.plot(xdata, yfit, color="orange", label="Fit")
ax.fill_between(
    xdata,
    yfit + yfit_err,
    yfit - yfit_err,
    color="orange",
    alpha=0.12,
    zorder=0,
    label="Fit Error",
)

# plot annotations
plt.legend(fontsize=14, loc="upper left")

txt = f"$f(x) = {explin.latex_str}$\n" f"$r^2 = {explin.rsq:.3f}$\n"
for name, param, err in zip(explin.param_names, explin.params, explin.param_errors):
    txt += f"{name} = {param:.3f} $\\pm$ {err:.3f}\n"
txt_loc = [-13.0, ax.get_ylim()[1]]
txt_loc = ax.transAxes.inverted().transform(ax.transData.transform(txt_loc))
txt_loc[0] -= 0.02
txt_loc[1] -= 0.05
ax.text(
    txt_loc[0],
    txt_loc[1],
    txt,
    fontsize="large",
    transform=ax.transAxes,
    va="top",
    linespacing=1.5,
)

### Root solving

[fsolve()]: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fsolve.html
[ExponentialPlusLinear().root_solve()]: ../../api/plasmapy.analysis.fit_functions.ExponentialPlusLinear.rst#plasmapy.analysis.fit_functions.ExponentialPlusLinear.root_solve
[Linear().root_solve()]: ../../api/plasmapy.analysis.fit_functions.Linear.rst#plasmapy.analysis.fit_functions.Linear.root_solve

An exponential plus a linear offset has no analytical solutions for its roots, except for a few specific cases.  To get around this, [ExponentialPlusLinear().root_solve()] uses SciPy's [fsolve()] to calculate it's roots.  If a fit function has analytical solutions to its roots (e.g. [Linear().root_solve()]), then the method is overridden with the known solution.

In [ ]:
root, err = explin.root_solve(-15.0)
(root, err)

[Linear().root_solve()]: ../../api/plasmapy.analysis.fit_functions.Linear.rst#plasmapy.analysis.fit_functions.Linear.root_solve

Let's use [Linear().root_solve()] as an example for a known solution.

In [ ]:
lin = ffuncs.Linear(params=(1.0, -5.0), param_errors=(0.1, 0.1))
root, err = lin.root_solve()
(root, err)